In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from skimage import io, transform
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import time
from tqdm import tqdm, trange

import warnings
warnings.filterwarnings("ignore")

root_dir = "./Participants_Data_TWC/"
csv_file = root_dir+'train.csv'

In [2]:
dataset = pd.read_csv(csv_file)

In [3]:
class WaterMarkingDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, root, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root
        self.transform = transform
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        img_name = os.path.join(self.root_dir+self.dataset.iloc[index, 0])
        img = io.imread(img_name)
        label = self.dataset.iloc[index, 1]
        label = np.array([label])
        if self.transform:
            img = self.transform(img)
        return {"image":img,"label":label}
        

In [4]:
# transform=transforms.Compose(
#         [transforms.Resize(32),transforms.ToTensor()]
#     ),
watermark_dataset = WaterMarkingDataset(
    "./Participants_Data_TWC/train.csv", 
    './Participants_Data_TWC/train/',
    transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize(256),transforms.ToTensor() ])
)

In [5]:
# fig = plt.figure()
# for i in range(len(watermark_dataset)):
#     sample = watermark_dataset[i]
#     print(i, sample['image'].shape, sample['label'].shape)

#     ax = plt.subplot(1, 4, i + 1)
#     plt.tight_layout()
#     ax.set_title('Sample #{}'.format(i))
#     ax.axis('off')
#     plt.imshow(sample['image'])

#     if i == 3:
#         plt.show()
#         break

In [6]:
batch_size = 64
validation_split = .2
shuffle_dataset= True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(watermark_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(watermark_dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(watermark_dataset, batch_size=batch_size,sampler=valid_sampler)


In [7]:
class BinaryImageClassifier(nn.Module):
    def __init__(self):
        super(BinaryImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32*126*126, 256)
        self.fc2 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
#         x = torch.permute(x, (0,3,2,1))
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(nn.functional.relu(self.conv2(x)), 2)
#         print(x.shape)
        x = x.view(-1, 32*126*126)
        x = nn.functional.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        
        return x

In [8]:
def save_network(epoch_label, model, optimizer, calculate_loss):
    save_filename = 'net_%s.pth' % epoch_label
    save_filename = 'net_%s.pth' % epoch_label
    save_path = os.path.join('./phase_4', save_filename)
    torch.save({'epoch': epoch_label,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'loss': calculate_loss,}, save_path)

    
def fit(train_loader, model, loss_function, optimizer, device):
    model=model.to(device)
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for i, batch in enumerate(tqdm(train_loader, desc="Training",bar_format='{desc:<5.5}{percentage:4.0f}%|{bar:10}{r_bar}') ):
        optimizer.zero_grad()
        images = batch['image'].to(device)
        target = batch['label'].to(device)
        
        output = model(images.float())
        loss = loss_function(output, target.float())
                                   
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
        predicted = (output > 0.5).float()
        total += target.size(0)
#         print("====>", predicted,target)
        correct += (predicted == target).sum().item()
#         print("correct, total :", correct, total)
    train_loss /= len(train_loader)
    train_accuracy = correct / total
    return train_loss, train_accuracy

In [9]:
def evaluate(val_loader, model, loss_function, optimizer, device):
    model=model.to(device)
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    for i, batch in enumerate(tqdm(val_loader, desc="validating", bar_format='{desc:<5.5}{percentage:4.0f}%|{bar:10}{r_bar}')):
        images = batch['image'].to(device)
        target = batch['label'].to(device)
        
        output = model(images.float())
        loss = loss_function(output, target.float())
        
        val_loss += loss.data.item()*images.size(0)
        predicted = (output > 0.5).float()
        total += target.size(0)
        correct += (predicted == target).sum().item()
    val_loss /= len(val_loader)
    val_accuracy = correct / total
    return val_loss, val_accuracy
        




In [10]:
def train(train_loader,val_loader, model, loss_function, optimizer, epochs, device):
    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
            optimizer.param_groups[0]['lr'], epochs, device)
        )
    start_time = time.time()
    history = {}
    history['train_loss'] = []
    history['val_loss'] = []
    history['train_acc'] = []
    history['val_acc'] = []
    for epoch in range(epochs):
        train_loss, train_acc = fit(train_loader, model, loss_function, optimizer, device)
        val_loss, val_acc = evaluate(val_loader, model, loss_function, optimizer, device)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        save_network(epoch_label=epoch, model=model, optimizer=optimizer, calculate_loss=loss_function)
        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {train_loss:.4f}, Training Acc: {train_acc:.4f}, Val Loss:{val_loss:.4f}, Val Acc:{val_acc:.4f}")
    total_time_sec     =  time.time() - start_time
    time_per_epoch_sec = total_time_sec / epochs
    print()
    print('Time total:     %5.2f sec' % (total_time_sec))
    print('Time per epoch: %5.2f sec' % (time_per_epoch_sec))
    return history



In [11]:
device = "cuda" if torch.cuda.is_available () else "cpu"
model = BinaryImageClassifier()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs=25
history = train(train_loader, validation_loader, model, criterion, optimizer, epochs, device)

train() called: model=BinaryImageClassifier, opt=Adam(lr=0.001000), epochs=25, device=cpu



valid 100%|██████████| 50/50 [01:08<00:00,  1.37s/it]


Epoch [1/25], Training Loss: 2766.7169, Training Acc: 0.5630, Val Loss:2826.0000, Val Acc:0.5565


valid 100%|██████████| 50/50 [01:04<00:00,  1.30s/it]


Epoch [2/25], Training Loss: 2779.5000, Training Acc: 0.5639, Val Loss:2826.0000, Val Acc:0.5565


...

Epoch [6/25], Training Loss: 2779.5000, Training Acc: 0.5639, Val Loss:2826.0000, Val Acc:0.5565


valid 100%|██████████| 50/50 [01:11<00:00,  1.44s/it]


Epoch [7/25], Training Loss: 2779.5000, Training Acc: 0.5639, Val Loss:2826.0000, Val Acc:0.5565


Train   6%|▋         | 13/200 [00:55<13:25,  4.31s/it]


KeyboardInterrupt: 